In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

## Creating a specification and model
The following were the specification that I set before building and traning the model:

Variable 'text' needs to be define first which is achieved from web_scraping.ipynb

In [7]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

embedding_dim = 256 # The embedding dimension
epochs = 50 
seq_length = 200 # Sequence length
vocab = sorted(set(text))
examples_per_epoch = len(text)//seq_length
vocab_size = len(vocab) # Length of the vocabulary in chars
rnn_units = 1024 # Number of RNN units
steps_per_epoch = examples_per_epoch//BATCH_SIZE

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [9]:
# Create training examples / targets
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.LSTM, recurrent_activation='sigmoid')  #Use LSTM

### Constructing model

To construct the model, I use embedding as my input layer. I added two LSTM layers as well as several dropouts. For the output layer, I use a dense layer.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    
    tf.keras.layers.Dropout(0.2),
      
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      
    tf.keras.layers.Dropout(0.2),

    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(vocab_size)  
    ])

  return model

In [14]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  #print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15872     
_________________________________________________________________
dropout (Dropout)            (64, None, 256)           0         
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
dropout_2 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense (Dense)                (64, None, 62)            6

In [17]:
#sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) # TF 1.12
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 200, 62)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1274695


In [19]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Training the model

In [21]:
EPOCHS=50 # also try 70 epochs

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/50
9/9 [==============================] - 3s 325ms/step - loss: 4.5539
Epoch 2/50
9/9 [==============================] - 2s 202ms/step - loss: 3.5561
Epoch 3/50
9/9 [==============================] - 2s 199ms/step - loss: 3.1388
Epoch 4/50
9/9 [==============================] - 2s 202ms/step - loss: 3.0059
Epoch 5/50
9/9 [==============================] - 2s 202ms/step - loss: 2.8392
Epoch 6/50
9/9 [==============================] - 2s 195ms/step - loss: 2.6771
Epoch 7/50
9/9 [==============================] - 2s 199ms/step - loss: 2.5254
Epoch 8/50
9/9 [==============================] - 2s 197ms/step - loss: 2.4021
Epoch 9/50
9/9 [==============================] - 2s 200ms/step - loss: 2.3239
Epoch 10/50
9/9 [==============================] - 2s 197ms/step - loss: 2.2547
Epoch 11/50
9/9 [==============================] - 2s 199ms/step - loss: 2.1945
Epoch 12/50
9/9 [==============================] - 2s 198ms/step - loss: 2.1388
Epoch 13/50
9/9 [==============================] 

In [22]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_50'

In [23]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
dropout_3 (Dropout)          (1, None, 256)            0         
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dropout_4 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
dropout_5 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 62)            